# Hubbard Model

I'm planning on running VQE and Trotter simulation on a Hubbard Hamiltonian to better understand its structure. 

The Hubbard model is a simplification of the interactions of electrons in a solid. It can be used to explain how these interactions yield insulating, magnetic, and superconducting effects (though I don't understand any of this yet). 

**Update 1**: I started by defining my own classes/functions for creation/annihilation operators, the Jordan-Wigner transform, the square lattice for the Hubbard Hamiltonian, the Hubbard Hamiltonian's matrix representation, and the Variational Hamiltonian Ansatz (VHA). As expected, my code and computer's RAM wasn't enough to work with anything more than a 2x2 square lattice (which has 8 qubits, so exists 256-dimensional Hilbert space). What did surprise me was how much more efficient OpenFermion was, because when I tried it when SSHing onto a Google Cloud computer, I was able to quickly solve the ground state of up to a 2x10 square lattice Hubbard Hamiltonian. Most of the functions/classes I defined are still available in the tools/ folder if you want to take a look. 

In [1]:
import numpy as np
import scipy.linalg
import scipy.optimize 

from tools.utils import *
tol = 0.005 # Tolerance for elementwise equality of matrices

$\newcommand{\ket}[1]{\lvert #1 \rangle}$
$\newcommand{\bra}[1]{\langle #1 \rvert}$

### Defining creation and annihilation operators 

Since we're describing electrons, the Hubbard Hamiltonian's creation and annihilation operators have the normal fermionic anticommutation relations, namely that for a fermion on site $j,k$ with spin $\sigma, \pi$, 

$$ \large \{ c_{j \sigma}, c^\dagger_{k \pi } \} = \delta_{jk} \delta_{\sigma \pi} \qquad \{ c^\dagger_{j\sigma}, c^\dagger_{k \pi} \}= \{ c_{j\sigma} c_{k\pi} \} = 0  $$

#### Some motivation for these relations 

I don't think there's a very simple explanation of why these relations hold. Even [Feynman admits](https://www.feynmanlectures.caltech.edu/III_04.html) he doesn't have a simple explanation for the mathematical differences for fermions and bosons. Nonetheless, we'll try to extend our knowledge of the quantum harmonic oscillator to fermions. 

Let's start with the latter relation. Fermions are identical particles, so we don't care about how individual particles move and behave, rather choosing to fixate on the properties of the whole. This is why in second quantization, we work in a Fock space. In Fock space, we assign each spin-orbital either $\ket{0}$ if it doesn't contain an electron or $\ket{1}$ if it does. 

From QHO, we know $a^\dagger$ is the creation operator and $a$ is the annihilation operator. We want some relation that makes it clear that $a^2$ and ${a^\dagger}^2$ are both nonsense. Since we're working in a vector space, we can set these both to equal 0, which has the effect of destroying our whole vector space. Notice another way of writing $a^2 = 0$, is $2 a^2 = \{ a, a \} = 0$, which is the anticommutation relation we have. The same logic works for $a^\dagger$. 

Recall from QHO that $a \ket{n} = \sqrt{n} \ket{n-1}$ and $a^\dagger \ket{n} = \sqrt{n + 1} \ket{n+1}$. Let's enforce the rule that $n$ can be only 0 or 1. Now, using the fact that $\sqrt{0} = 0$ and $\sqrt{1} = 1$, we can write $a \ket{n} = n \ket{1-n}$ and $a^\dagger \ket{n} = (1 - n) \ket{1-n}$. 

Now, what is $(a a^\dagger + a^\dagger a) \ket{n}$? It simplifies to $(2n^2 - 2n + 1) \ket{n}$. Plugging in $n = 0$ and $n=1$ results in the same thing: $2n^2 -2n + 1 = 1$. So we can write that $\{ a, a^\dagger \} = 1$. 

### Jordan-Wigner Transformation

We let $\mid 0 \rangle$ represent no electron in a spin-orbital, and we let $\mid 1 \rangle$ represent an electron existing in a spin-orbital. This allows us to define the creation operator as $(X-iY)/2$ and the annihilation operator as $(X+iY)/2$. The main problem with this is that the creation and annihilation operators don't anti-commute. To fix this, we prepend the operator with Pauli-$Z$ tensors. Because the Pauli operators anti-commute, this preserves our desired anti-commutation relation. 

## Defining the Hubbard Hamiltonian

Okay, now that we have those preliminaries out of the way, what is the Hubbard Hamiltonian? What's the motivation behind it? How is it useful? 

Answer these. 

The Hubbard Hamiltonian: 
$$ \large H = -t \sum_{<j, k>, \sigma} \Big( c^\dagger_{j\sigma} c_{k\sigma} + c^\dagger_{k \sigma} c_{j \sigma} \Big) + U \sum_j n_{j \uparrow} n_{j \downarrow} - \mu \sum_j \Big(n_{j\uparrow} + n_{j\downarrow} \Big) $$

The first term is kinetic energy, a fermion moving from one site to another. The symbol $<j, k>$ implies iterating over sites that are adjacent. 

The second term is interaction energy, additional energy for a doubly-occupied site. 

The third term is chemical potential, which controls the filling. 

**TODO: Is my model getting this final term? It should be fine, because it's still just a quadratic term so we can efficiently find the ground state.**

### 2D Hubbard Hamiltonian on a Square Lattice 

Here is a model of a 2D Hubbard Hamiltontian on a square lattice from [1811.04476](https://arxiv.org/pdf/1811.04476.pdf). 

![](images/2d_hubbard.png)

We'll use OpenFermion's `HubbardSquareLattice` class to define our lattice. OpenFermion has a simpler function `fermi_hubbard()` to create a `FermionOperator` to describe our system, but this doesn't give us access to the specific hopping terms in the Hamiltonian (the terms in the first summation above), which we'll need for the Variational Hamiltonian Ansatz. 

The `HubbardSquareLattice` class has a useful method: `site_pair_iter(edge_type)`. We'll use the 'horizontal_neighbor' and 'vertical_neighbor' edge types. But we also need to differentiate between "even" and "odd" horizontal and vertical neighbors: even horizontal neighbors will be horizontal neighbors whose leftmost site has even index, and likewise for horizontal odd, vertical even, and vertical odd neighbors. To get this added specificity, we'll need to examine each item in the iterable generated by `site_pair_iter()`. We need the even and odd terms because then when we Trotterize the Hubbard Hamiltonian, we don't introduce any Trotter error, because the our four sets of tunneling terms (even horizontal, even vertical, odd horizontal, odd vertical) commute with each other. 

Actually, it will be much easier to just subclass `HubbardSquareLattice` and define our own `site_pair_iter()` that allows us to specify 'even' or 'odd'. I'll call this class `DecomposedHubbardSquareLattice`.

In [6]:
from openfermion.utils import commutator 
from openfermion.ops import FermionOperator 
from openfermion.transforms import jordan_wigner 

tunneling_01 = FermionOperator('0^ 1') + FermionOperator('1^ 0')
tunneling_12 = FermionOperator('1^ 2') + FermionOperator('2^ 1')
tunneling_23 = FermionOperator('2^ 3') + FermionOperator('3^ 2')

print('Commutator of 01 and 12 is: ', commutator(tunneling_01, tunneling_12))
print('\nJW matrix of above commutator is: ', commutator(jordan_wigner(tunneling_01), jordan_wigner(tunneling_12)))
print('\nCommutator of 01 and 23 is: ', commutator(tunneling_01, tunneling_23))
print('\nJW matrix of above commutator is: ', commutator(jordan_wigner(tunneling_01), jordan_wigner(tunneling_23)))

Commutator of 01 and 12 is:  1.0 [0^ 1 1^ 2] +
1.0 [0^ 1 2^ 1] +
1.0 [1^ 0 1^ 2] +
1.0 [1^ 0 2^ 1] +
-1.0 [1^ 2 0^ 1] +
-1.0 [1^ 2 1^ 0] +
-1.0 [2^ 1 0^ 1] +
-1.0 [2^ 1 1^ 0]

JW matrix of above commutator is:  0.5j [X0 Z1 Y2] +
-0.5j [Y0 Z1 X2]

Commutator of 01 and 23 is:  1.0 [0^ 1 2^ 3] +
1.0 [0^ 1 3^ 2] +
1.0 [1^ 0 2^ 3] +
1.0 [1^ 0 3^ 2] +
-1.0 [2^ 3 0^ 1] +
-1.0 [2^ 3 1^ 0] +
-1.0 [3^ 2 0^ 1] +
-1.0 [3^ 2 1^ 0]

JW matrix of above commutator is:  0


In [8]:
from openfermion.utils import HubbardSquareLattice
#HubbardSquareLattice?

**PROBLEM:** I noticed the source code for how OpenFermion creates the iterator over the neighbors in a Hubbard lattice is that it allows sites at the end to loop around, ie the last site has a tunnelling term with the first site. This is a problem, because 
1. I didn't know this was the case and should look into it more. 
2. I usually write the code for even and odd sites to not loop back to the start. 

I guess this isn't that big of a problem; I can just change my code to loop back to the start. I'll need to do the same checking if `x_dimension` or `y_dimension` are less than 2. 

First, let me check if the Hamiltonians are iso-spectral. I don't think they will be. Wait is this what periodic means?

In [21]:
# HubbardSquareLattice parameters
x_n = 2 
y_n = 2 
n_dofs = 1 # 1 degree of freedom for spin, this might be wrong. Having only one dof means ordered=False. 
periodic = 0 # Not sure what this is, periodic boundary conditions?
spinless = 0 # Has spin

import itertools
class DecomposedHubbardSquareLattice(HubbardSquareLattice):
    @property 
    def edge_types(self):
        # Overriding edge_types property so we can define additional ones 
        return ('onsite', 'neighbor', 'diagonal_neighbor', 'horizontal_neighbor', 'vertical_neighbor', 
                'hor_even_neighbor', 'hor_odd_neighbor', 'ver_even_neighbor', 'ver_odd_neighbor')
        
    def site_pairs_iter(self, edge_type, ordered=True):
        # `ordered` parameter just flips the order: if True, for each a,b -> (a,b), (b,a); if False, for each 
        # a,b -> (a,b), so we only get it once and it doesn't flip order. 
        # We WANT ordered=False, because there's a helper function tunneling_operator(i, j, coeff) that takes in 
        # two sites i,j and does coeff*(i^j j^i). 
        
        # Overriding site_pairs_iter() to add functionality for additional edge_types 
        if edge_type == 'onsite':
            return ((i, i) for i in self.site_indices)
        elif edge_type == 'neighbor':
            return self.neighbors_iter(ordered)
        elif edge_type == 'horizontal_neighbor':
            return self.horizontal_neighbors_iter(ordered)
        elif edge_type == 'vertical_neighbor':
            return self.vertical_neighbors_iter(ordered)
        elif edge_type == 'diagonal_neighbor':
            return self.diagonal_neighbors_iter(ordered)
        # Above was copied from utils._lattice.py so we handle old edge_types correctly. 
        # Below is added functionality for new edge_types. 
        elif edge_type == 'hor_even_neighbor':
            return self.hv_eo_neighbors(lambda x,y: 1-x%2, lambda x,y: (x+1, y), ordered) 
        elif edge_type == 'hor_odd_neighbor':
            return self.hv_eo_neighbors(lambda x,y: x%2, lambda x,y: (x+1, y), ordered)
        elif edge_type == 'ver_even_neighbor':
            return self.hv_eo_neighbors(lambda x,y: 1-y%2, lambda x,y: (x, y+1), ordered)
        elif edge_type == 'ver_odd_neighbor':
            return self.hv_eo_neighbors(lambda x,y: y%2, lambda x,y: (x, y+1), ordered)
        raise ValueError('Edge type {} is not valid.'.format(edge_type))
        
    def hv_eo_neighbors(self, filter_xy, map_next_xy, ordered=True):
        for i in range(self.x_dimension):
            for j in range(self.y_dimension):
                if filter_xy(i, j):
                    # Get indices for next site  
                    k, l = map_next_xy(i, j)
                    # Make sure next site isn't out of bounds 
                    if k >= self.x_dimension or l >= self.y_dimension: continue 
                    
                    site_a = self.to_site_index((i, j))
                    site_b = self.to_site_index((k, l))
                    
                    yield (site_a, site_b)
                    if ordered: yield (site_b, site_a)
        
    # Function OpenFermion uses for 'horizontal_neightbor' edge_type.  
    # PROBLEM: This loops around! Notice the % self.x_dimension. 
    def horizontal_neighbors_iter(self, ordered=True):
        n_horizontal_edges_per_y = (
            self.x_dimension - (self.x_dimension <= 2 or not self.periodic))
        for x in range(n_horizontal_edges_per_y):
            for y in range(self.y_dimension):
                i = self.to_site_index((x, y))
                j = self.to_site_index(((x + 1) % self.x_dimension, y))
                yield (i, j)
                if ordered:
                    yield (j, i)

    # Function OpenFermion uses for 'vertical_neighbor' edge_type
    def vertical_neighbors_iter(self, ordered=True):
        n_vertical_edges_per_x = (self.y_dimension -
                                  (self.y_dimension <= 2 or not self.periodic))
        for y in range(n_vertical_edges_per_x):
            for x in range(self.x_dimension):
                i = self.to_site_index((x, y))
                j = self.to_site_index((x, (y + 1) % self.y_dimension))
                yield (i, j)
                if ordered:
                    yield (j, i)
    
    # I'm changing this function so that it uses my functions, so I can compare the spectrums, since 
    # FermiHubbardModel.hamiltonian() will call this to generate the iterable. 
    def neighbors_iter(self, ordered=True):
        return itertools.chain(
            #self.horizontal_neighbors_iter(ordered),
            #self.vertical_neighbors_iter(ordered)
            
            # itertools.chain('ABC', 'DEF') = Iterable('ABCDEF') 
            self.site_pairs_iter('hor_even_neighbor', ordered), 
            self.site_pairs_iter('hor_odd_neighbor', ordered), 
            self.site_pairs_iter('ver_even_neighbor', ordered), 
            self.site_pairs_iter('ver_odd_neighbor', ordered), 
        )


lattice = DecomposedHubbardSquareLattice(x_n, y_n, n_dofs=n_dofs, periodic=periodic, spinless=spinless)

Now, we'll create a `FermiHubbardModel` instance by passing it our `HubbardSquareLattice` instance defined above. 

To get the `FermionOperator` instance, we need to call `FermiHubbardModel.hamiltonian()`. The [documentation](https://openfermion.readthedocs.io/en/latest/openfermion.html#openfermion.hamiltonians.FermiHubbardModel) isn't that great here, but the [source code](https://github.com/quantumlib/OpenFermion/blob/master/src/openfermion/hamiltonians/_general_hubbard.py) indicates we do indeed get a `FermionOperator` instance which we'll need for calculating ground state, etc. 

We can't just pass an integer for $t$, $U$, or $\mu$ in this class. Instead, we have to specify the *specific* coefficient for each pair and edge type ($t_{ij}^{\textrm{horizontal neighbor}}$). This will be useful later on, I think, because [1506.05135](https://arxiv.org/abs/1506.05135) says we'll need the indices for different values during adiabatic evolution. 

In [22]:
from openfermion.hamiltonians import FermiHubbardModel
#FermiHubbardModel??

In [23]:
from openfermion.utils import SpinPairs
tunneling = [('neighbor', (0, 0), 1.)] # Not sure if this is right
interaction = [('onsite', (0, 0), 2., SpinPairs.DIFF)] # Not sure if this is right
potential = None 
mag_field = 0. 
particle_hole_sym = False # Not sure if this is right

In [24]:
hubbard = FermiHubbardModel(lattice , tunneling_parameters=tunneling, interaction_parameters=interaction, 
                            potential_parameters=potential, magnetic_field=mag_field, 
                            particle_hole_symmetry=particle_hole_sym)

In [25]:
# In tunneling_terms(), there's the following line: 
#                a, aa = param.dofs
#                site_pairs = self.lattice.site_pairs_iter(param.edge_type, a != aa)
# which means we set ordered=False, if we have only one dof. This is wrong behavior right? 
# I think we can correct this by summing both i^j + j^i when we use the iterator.
# UPDATE: Scratch all that, they define and call tunneling_operator(i, j, coeff) to add both terms. 
hubbard.tunneling_parameters[0].dofs

(0, 0)

In [26]:
# Test: Check if the above Hamiltonian is iso-spectral with a Hubbard Hamiltonian with same parameters 
from openfermion.hamiltonians import fermi_hubbard
from openfermion.utils import eigenspectrum
test_hub = fermi_hubbard(x_n, y_n, 1. , 2., chemical_potential=0.0, magnetic_field=mag_field, 
                         periodic=periodic, spinless=spinless, particle_hole_symmetry=particle_hole_sym)
print("Spectrums are the same: ", array_eq(eigenspectrum(test_hub), eigenspectrum(hubbard.hamiltonian()), tol))

Spectrums are the same:  True


Huh. I didn't expect the spectrums to be the same since I removed the loop-back tunneling. 
**Does having same spectrum mean they're the same operator?**

In [27]:
from openfermion.ops import FermionOperator

def tunneling_operator(i, j, coefficient=1.):
    # Copied from hamiltonians/_lattice.py in OpenFermion
    return (FermionOperator(((i, 1), (j, 0)), coefficient) + FermionOperator(
        ((j, 1), (i, 0)), coefficient.conjugate()))
def tunneling_terms_hor_even(hor, even, model):
    # Mostly copied from FermiHubbardMode.tunneling_terms() 
    terms = FermionOperator()
    for param in model.tunneling_parameters:
        a, aa = param.dofs 
        # We don't use param.edge_type because it's 'neighbor' and we need to be more specific
        if hor and even:
            site_pairs = model.lattice.site_pairs_iter('hor_even_neighbor', a != aa)
        elif hor and not even: 
            site_pairs = model.lattice.site_pairs_iter('hor_odd_neighbor', a != aa)
        elif not hor and even: 
            site_pairs = model.lattice.site_pairs_iter('ver_even_neighbor', a != aa)
        elif not hor and not even:
            site_pairs = model.lattice.site_pairs_iter('ver_odd_neighbor', a != aa)

        for r, rr in site_pairs: 
            for spin_index in model.lattice.spin_indices:
                i = model.lattice.to_spin_orbital_index(r, a, spin_index)
                j = model.lattice.to_spin_orbital_index(rr, aa, spin_index)
                terms += tunneling_operator(i, j, -param.coefficient)
    return terms

In [28]:
# All the values in this dictionary are instances of FermionOperator
hamiltonians = {
    'hub': hubbard.hamiltonian(), 
    'non_interacting': hubbard.tunneling_terms(), 
    'hor_even': tunneling_terms_hor_even(True, True, hubbard), 
    'hor_odd': tunneling_terms_hor_even(True, False, hubbard), 
    'ver_even': tunneling_terms_hor_even(False, True, hubbard), 
    'ver_odd': tunneling_terms_hor_even(False, False, hubbard)
}

In [29]:
# Check to make sure summing up the parts gives us the whole of non_interacting term 
print('Sum of horizontal/vertical even/odd terms gives non-interacting term: ', 
      (hamiltonians['hor_even'] + hamiltonians['hor_odd'] + hamiltonians['ver_even'] + 
       hamiltonians['ver_odd'] == hamiltonians['non_interacting']))

Sum of horizontal/vertical even/odd terms gives non-interacting term:  True


## Variational Hamiltonian Ansatz

The VHA is an ansatz inspired by time-evolution of the system. It splits the Hamiltonian into sub-operators and then does time-evolution for those operators: 
$$\large U(\theta) = \prod_{k=1}^n \prod_{\alpha=1}^N \exp \Big( i\theta_{\alpha, k} H_{\alpha} \Big) $$
where $H_\alpha$ are the sub-Hamiltonians and $\theta$ are the parameters being optimized. 

**Question:** Why is this a good ansatz? It seems it only has access to states that are evolutions of the state we start with. Why is that a guarantee that it'll approximate the ground state? 

**Answer:** It's based on adiabatic evolution. I don't understand this but somehow if we start by evolving the ground state of a Hamiltonian (non-interacting part in this case) and then slowly start replacing it with another Hamiltonian (full Hubbard Hamiltonian), we get the corresponding ground state of the new Hamiltonian by magic. 

In [1811.04476](https://arxiv.org/pdf/1811.04476.pdf) they use $N=5$, splitting it as I did above: even and odd horizontal hopping terms, even and odd vertical horizontal terms, and the on-site interaction terms. 

The non-interacting term is quadratic so we can use `openfermioncirq.prepare_gaussian_state` to get its eigenstate. I can basically follow the first part of Tutorial 4 in openfermion-cirq. 

In [30]:
# Compute ground state on GCloud computers

from openfermion import get_sparse_operator, get_ground_state
hub_sparse = get_sparse_operator(hamiltonians['hub'])
# genergy, gstate = get_ground_state(hub_sparse)
# print("Ground state energy: ", genergy)

# "Ground state energy: -3.6272130052966762" 

How can we implement the above exponentials in a quantum circuit? Hm... 

OF has a `SwapNetworkTrotterHubbard` ansatz. How does it work? 

Well, the SwapNetwork uses only `ISWAP`, `PhasedISWAP`, `CZ` and `Z` gates. What does it do? 

It was proposed in arxiv: 1711.04789. It allows us to simulate a Trotter step in $N$ depth and $N^2 / 2$ two-qubit entangling gates, and lets us prepare arbitrary Slater determinants in $N/2$ depth, all assuming only a linearly connected architecture. 

In [31]:
from openfermion.transforms import get_diagonal_coulomb_hamiltonian
from openfermioncirq import HamiltonianObjective, SwapNetworkTrotterAnsatz

# Convert to DiagonalCoulombHamiltonian 
# I think this is because the ansatz uses properties of this?
dc_hub = get_diagonal_coulomb_hamiltonian(hamiltonians['hub'])
# Define objective function as Hamiltonian averaging
obj = HamiltonianObjective(dc_hub)
# Create swap network Trotter ansatz for linear time (need to read this paper)
steps = 5
# What's the difference between this and SwapNetworkTrotterHubbardAnsatz
ansatz = SwapNetworkTrotterAnsatz(dc_hub, iterations=steps)

print('Our ansatz circuit:')
print(ansatz.circuit.to_text_diagram(transpose=True))

Our ansatz circuit:
0     1                2                3         4  5         6                7
│     │                │                │         │  │         │                │
@─────@^V_0_1_0        @────────────────@^V_2_3_0 @──@^V_4_5_0 @────────────────@^V_6_7_0
│     │                │                │         │  │         │                │
×ᶠ────×ᶠ               ×ᶠ───────────────×ᶠ        ×ᶠ─×ᶠ        ×ᶠ───────────────×ᶠ
│     │                │                │         │  │         │                │
│     ×ᶠ───────────────×ᶠ               ×ᶠ────────×ᶠ ×ᶠ────────×ᶠ               │
│     │                │                │         │  │         │                │
iSwap─iSwap^(-T_1_3_0) ×ᶠ───────────────×ᶠ        ×ᶠ─×ᶠ        iSwap────────────iSwap^(-T_4_6_0)
│     │                │                │         │  │         │                │
×ᶠ────×ᶠ               │                ×ᶠ────────×ᶠ │         ×ᶠ───────────────×ᶠ
│     │                │                │         │  

In [44]:
from openfermioncirq import SwapNetworkTrotterHubbardAnsatz
steps = 5
ansatz = SwapNetworkTrotterHubbardAnsatz(x_n, y_n, 1., 2., periodic=False, iterations=steps)

print('Our ansatz circuit:') 
print(ansatz.circuit.to_text_diagram(transpose=True))

Our ansatz circuit:
0     1             2     3             4     5             6     7
│     │             │     │             │     │             │     │
iSwap─iSwap^(-Th_0) iSwap─iSwap^(-Th_0) iSwap─iSwap^(-Th_0) iSwap─iSwap^(-Th_0)
│     │             │     │             │     │             │     │
×ᶠ────×ᶠ            ×ᶠ────×ᶠ            ×ᶠ────×ᶠ            ×ᶠ────×ᶠ
│     │             │     │             │     │             │     │
│     ×ᶠ────────────×ᶠ    ×ᶠ────────────×ᶠ    ×ᶠ────────────×ᶠ    │
│     │             │     │             │     │             │     │
iSwap─iSwap^(-Tv_0) ×ᶠ────×ᶠ            ×ᶠ────×ᶠ            iSwap─iSwap^(-Tv_0)
│     │             │     │             │     │             │     │
×ᶠ────×ᶠ            │     ×ᶠ────────────×ᶠ    │             ×ᶠ────×ᶠ
│     │             │     │             │     │             │     │
│     @─────────────@^V_0 │             │     @─────────────@^V_0 │
│     │             │     │             │     │             │     │
│ 

In [45]:
from openfermioncirq import prepare_gaussian_state, VariationalStudy

# VariationalStudy??

In [46]:
from openfermion import QuadraticHamiltonian
from cirq import Circuit
from openfermioncirq import prepare_gaussian_state, VariationalStudy

# Initial state |v_0> 
n_electrons = x_n * y_n # Half fill
prep_circ = Circuit(
    prepare_gaussian_state(
        # We choose start state to be ground state of tunneling terms
        # But isn't this degenerate?
        ansatz.qubits, 
        QuadraticHamiltonian(dc_hub.one_body), 
        ))

# Create a Hamiltonian variational study 
study = VariationalStudy(
    'Hubbard-VHA', 
    ansatz, 
    obj, 
    preparation_circuit=prep_circ)

print('Created a variational study with {} qubits and {} parameters'.format(
    len(study.ansatz.qubits), study.num_params))

print("The value of the objective with default initial parameters is {}".format(
    study.value_of(ansatz.default_initial_params())))

print("The circuit of the study is:")
print(study.circuit.to_text_diagram(transpose=True))

Created a variational study with 8 qubits and 15 parameters
The value of the objective with default initial parameters is -2.8669747741856813
The circuit of the study is:
0             1                   2                    3                    4                     5                    6                    7
│             │                   │                    │                    │                     │                    │                    │
X             X                   X                    │                    │                     │                    │                    │
│             │                   │                    │                    │                     │                    │                    │
│             │                   PhISwap(0.25)────────PhISwap(0.25)^0.706  │                     │                    │                    │
│             │                   │                    │                    │                     │                    

In [47]:
# We have 60 parameters. What are they? 
# They're in format (U/T/W/V, p, q, i) where p,q are qubits and i is iteration
# Seems 12 parameters per iteration. Per iteration, 4 interaction terms like we'd expect, 
# and then 8 tunneling terms like we'd expect. 

# How can I make these fewer? YO, SwapNetworkTrotterHubbard has only 3 parameters per iteration!
list(ansatz.params())

[ofc.variational.letter_with_subscripts.LetterWithSubscripts('Th', 0),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('Tv', 0),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('V', 0),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('Th', 1),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('Tv', 1),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('V', 1),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('Th', 2),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('Tv', 2),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('V', 2),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('Th', 3),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('Tv', 3),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('V', 3),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('Th', 4),
 ofc.variational.letter_with_subscripts.LetterWithSubscripts('Tv', 4),
 ofc.varia

In [67]:
from openfermioncirq.optimization import ScipyOptimizationAlgorithm, OptimizationParams

# Optimize
algorithm = ScipyOptimizationAlgorithm(
    kwargs={'method': 'Powell'},
    options={'maxiter': 500},
    uses_bounds=True) 
# Try with True? If that doesn't improve, save parameters and see what fidelity is. Then, try adding 
# more iterations. 
optimization_params = OptimizationParams(
    algorithm=algorithm)
result = study.optimize(optimization_params)
print(result.optimal_value)

/home/warren/github/qc-mentorship/env/lib/python3.6/site-packages/scipy/optimize/_minimize.py:522: RuntimeWarning: Method Powell cannot handle constraints nor bounds.
  RuntimeWarning)


-3.0262376910117164


In [49]:
study.trial_results

OrderedDict([(0,
              <openfermioncirq.optimization.result.OptimizationTrialResult at 0x7f5928069198>)])

In [55]:
#study.extend_result(0)

In [57]:
from openfermion import QubitOperator
isinstance(dc_hub, QubitOperator)

False

In [65]:
obj.variance_bound

196.0

In [58]:
np.sqrt(196)

14.0

In [60]:
qubit_op = jordan_wigner(dc_hub)
qubit_op.constant

2.0

### Old VHA implementation

In [ ]:
def time_evo(ham, t): 
    # returns e^{i*t*ham}
    # scipy.linalg.expm() uses the Pade approximant; try to understand more of how it works because it seems 
    # pretty useful: https://en.wikipedia.org/wiki/Pad%C3%A9_approximant
    return scipy.linalg.expm(-1j * t * ham)

def prod_time_evo(ham_sets, ts):
    result = 1 
    for pair in zip(ham_sets, ts): 
        result *= time_evo(pair[0], pair[1])
    return result

def test_linalg_expm():
    return scipy.linalg.expm(X)

test_linalg_expm()

In [ ]:
scipy.linalg?

In [ ]:
y = [1, 1]
2 * y

In [ ]:
class VHA: 
    def __init__(self, hubbard, start_state): 
        self.ham_sets = hubbard.ham_sets 
        self.hamiltonian = hubbard.hamiltonian
        self.start_state = start_state
        self.start_params = np.array([5e30 for i in range(5)])
            
    def optimize(self):  
        self.energies = []
        
        def energy(t_params): 
            unitary = prod_time_evo(2 * self.ham_sets, np.real(t_params))
            state = np.dot(unitary, self.start_state)
            curr_energy = np.dot(np.transpose(np.conj(state)), np.dot(self.hamiltonian, state))[0][0]
            #print(curr_energy)
            self.energies.append(curr_energy)
            return curr_energy
                    
        self.sol = scipy.optimize.minimize(energy, 2 * self.start_params, method='Powell', 
                                           bounds=[(0, 3) for i in range(2 * len(self.start_params))])

In [ ]:
vha = VHA(x, gstate)

In [ ]:
vha.optimize()

In [ ]:
print('Minimum energy: ', vha.sol.fun)
print('Number of iterations: ', vha.sol.nfev)
est_g_state = np.dot(prod_time_evo(vha.ham_sets, vha.sol.x), vha.start_state)
print('Estimated ground state: ', est_g_state)
print(vha.sol.x)

# Doing it without OpenFermion-Cirq

### Finding Ground State of Non-interacting Terms

VHA works because finding the ground state of non-interacting terms is efficiently computable on a QC and the VHA can evolve from that ground state to the ground state of the entire Hubbard Hamiltonian. 

We'll start by finding the ground state by using numpy to see if this actually work. 

In [12]:
from openfermion.transforms import get_sparse_operator 
from openfermion.utils import get_ground_state 

sparse_op = get_sparse_operator(hamiltonians['hub'])
#g_energy, g_state = get_ground_state(sparse_op)
print("My kernel dies when trying to calculate. ")

My kernel dies when trying to calculate. 


In [16]:
prepare_gaussian_state??